## Overview

This analysis involves running the REopt API with two distinct scenarios:
- **Scenario 1: PV only** - Incorporates only a photovoltaic (PV) system.
- **Scenario 2: PV + BESS** - Includes both a PV system and a Battery Energy Storage System (BESS).

The API responses are stored as JSON files in the outputs folder for subsequent processing.


## Note:
Run this section of code first (before running the rest) and copy the Run UUIDs provided at the end as inputs within the main script as `run_uuid_1` and `run_uuid_2`

In [55]:
import pandas as pd
import json
import requests
import time
import os
import uuid
from collections import defaultdict
import re

API_KEY = "gYV8t7d6c5naotp67meIJyJRi6DksKv0VfPSQzEa"  # Replace with your API key
# Define the API key and URL
root_url = "https://developer.nrel.gov/api/reopt/stable"
# following is not necessary but silences warnings:
# InsecureRequestWarning: Unverified HTTPS request is being made to host 'developer.nrel.gov'. Adding certificate verification is strongly advised.
import urllib3
urllib3.disable_warnings()

Code to run two scenarios to get run_uuid for results for camparison

In [56]:
# inputs_path = os.path.join(".", 'inputs')
# outputs_path = os.path.join(".", 'outputs')

# # Define the post data for both scenarios
# post_1 = {
#   "Site": {
#     "latitude": 37.7749295,
#     "longitude": -122.4194155
#   },
#   "ElectricLoad": { "doe_reference_name": "Hospital" },
#   "ElectricTariff": { "urdb_label": "662aae965d742ded630a7e55" },
#   "PV": {
#     "array_type": 0,
#     "macrs_option_years": 5,
#     "macrs_bonus_fraction": 0.6,
#     "module_type": 0
#   },
#   "Wind": {
#     "macrs_option_years": 5,
#     "macrs_bonus_fraction": 0.6,
#     "size_class": "commercial",
#     "max_kw": 1000000000
#   },
#   "ElectricStorage": {
#     "macrs_option_years": 7,
#     "macrs_bonus_fraction": 0.6
#   },
#   "CHP": {
#     "prime_mover": "recip_engine",
#     "size_class": 2,
#     "fuel_type": "natural_gas",
#     "macrs_option_years": 5,
#     "macrs_bonus_fraction": 0.6,
#     "fuel_cost_per_mmbtu": 9.0
#   },
#   "DomesticHotWaterLoad": {
#     "annual_mmbtu": 6242.0,
#     "doe_reference_name": "Hospital"
#   },
#   "SpaceHeatingLoad": {
#     "annual_mmbtu": 1713.0,
#     "doe_reference_name": "Hospital"
#   },
#   "ExistingBoiler": {
#     "fuel_type": "natural_gas",
#     "production_type": "hot_water",
#     "fuel_cost_per_mmbtu": 9.0
#   }
# }

# post_2 = {
#   "Site": {
#     "latitude": 37.7749295,
#     "longitude": -122.4194155
#   },
#   "ElectricLoad": { "doe_reference_name": "Hospital" },
#   "ElectricTariff": { "urdb_label": "662aae965d742ded630a7e55" },
#   "PV": {
#     "array_type": 0,
#     "macrs_option_years": 5,
#     "macrs_bonus_fraction": 0.6,
#     "module_type": 0
#   },
#   "ElectricStorage": {
#     "macrs_option_years": 7,
#     "macrs_bonus_fraction": 0.6
#   },
#   "CHP": {
#     "prime_mover": "recip_engine",
#     "size_class": 2,
#     "fuel_type": "natural_gas",
#     "macrs_option_years": 5,
#     "macrs_bonus_fraction": 0.6,
#     "fuel_cost_per_mmbtu": 9.0
#   },
#   "DomesticHotWaterLoad": {
#     "annual_mmbtu": 6242.0,
#     "doe_reference_name": "Hospital"
#   },
#   "SpaceHeatingLoad": {
#     "annual_mmbtu": 1713.0,
#     "doe_reference_name": "Hospital"
#   },
#   "ExistingBoiler": {
#     "fuel_type": "natural_gas",
#     "production_type": "hot_water",
#     "fuel_cost_per_mmbtu": 9.0
#   }
# }

# # Function to POST inputs to the API
# def post_inputs_to_api(post_data, api_key, api_url):
#     headers = {'x-api-key': api_key}
#     response = requests.post(f"{api_url}/job/", headers=headers, json=post_data, verify=False)
#     response.raise_for_status()
#     return response.json()["run_uuid"]

# # Function to poll for results
# def poll_for_results(run_uuid, api_key, api_url, interval=5):
#     headers = {'x-api-key': api_key}
#     while True:
#         response = requests.get(f"{api_url}/job/{run_uuid}/results/", headers=headers, verify=False)
#         response.raise_for_status()
#         result = response.json()
#         if result["status"] != "Optimizing...":
#             return result
#         time.sleep(interval)



# # Post the inputs to the API and get the run_uuid for both scenarios
# run_uuid_1 = post_inputs_to_api(post_1, API_KEY, root_url)
# print(f"Run UUID for Scenario 1: {run_uuid_1}")

# run_uuid_2 = post_inputs_to_api(post_2, API_KEY, root_url)
# print(f"Run UUID for Scenario 2: {run_uuid_2}")

# # Poll for results for both scenarios
# results_1 = poll_for_results(run_uuid_1, API_KEY, root_url)
# print("Results retrieved successfully for Scenario 1.")

# results_2 = poll_for_results(run_uuid_2, API_KEY, root_url)
# print("Results retrieved successfully for Scenario 2.")

# # Save the results to JSON files
# outputs_file_name_1 = f"response_{run_uuid_1}.json"
# with open(os.path.join(outputs_path,outputs_file_name_1), 'w') as fp:
#     json.dump(results_1, fp, indent=4)
# print(f"Results saved to {outputs_file_name_1}")

# outputs_file_name_2 = f"response_{run_uuid_2}.json"
# with open(os.path.join(outputs_path,outputs_file_name_2), 'w') as fp:
#     json.dump(results_2, fp, indent=4)
# print(f"Results saved to {outputs_file_name_2}")


## Key Functions and Steps in the Analysis

### 1. `get_scenario_results`
Retrieves the results from the REopt API for a specified scenario using its unique run UUID. This function:
- Sends a GET request to the REopt API.
- Fetches the results in JSON format.

### 2. `process_scenarios`
Processes multiple scenarios to generate a consolidated DataFrame. Key steps include:
- **Data Extraction**: Utilizes the `get_REopt_data` function with a specified `config` to extract relevant data points from the API responses.
- **BAU Data Handling**: Checks and manages Business-As-Usual (BAU) values to ensure data consistency across scenarios.
- **Final DataFrame Creation**: Combines data from all scenarios into a single DataFrame and processes it for final output.

### 3. `get_REopt_data`
Extracts specific data points from the API response JSON based on a configuration (`config`). This function:
- Flattens the nested JSON structure.
- Retrieves values according to the configuration settings.
- Cleans and structures the data into a pandas DataFrame.

### 4. `generate_data_dict`
Generates a data dictionary based on the provided configuration and flattened data. This dictionary is then converted into a DataFrame for further analysis.


In [57]:
import xlsxwriter
import pandas as pd
import requests
import json
from collections import defaultdict
import re
import uuid

#### Helper Functions
def get_with_suffix(df, key, suffix, default_val=0):
    """Fetch value from dataframe with an optional retriaval of _bau suffix."""
    if not key.endswith("_bau"):
        key = f"{key}{suffix}"
    return df.get(key, default_val)

def flatten_dict(d, parent_key='', sep='.'):
    """Flatten nested dictionary."""
    items = []
    for k, v in d.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten_dict(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

def clean_data_dict(data_dict):
    """Clean data dictionary by removing default values."""
    for key, value_array in data_dict.items():
        new_value_array = [
            "" if v in [0, float("nan"), "NaN", "0", "0.0", "$0.0", -0, "-0", "-0.0", "-$0.0", None] else v
            for v in value_array
        ]
        data_dict[key] = new_value_array
    return data_dict

def sum_vectors(data):
    """Sum numerical vectors within a nested data structure."""
    if isinstance(data, dict):
        return {key: sum_vectors(value) for key, value in data.items()}
    elif isinstance(data, list):
        if all(isinstance(item, (int, float)) for item in data):
            return sum(data)
        else:
            return [sum_vectors(item) for item in data]
    else:
        return data

#### Core Functions
def generate_data_dict(config, df_gen, suffix):
    """Generate data dictionary based on configuration and dataframe."""
    data_dict = defaultdict(list)
    for var_key, col_name in config:
        if callable(var_key):
            val = var_key(df_gen)
        else:
            val = get_with_suffix(df_gen, var_key, suffix, "-")
        data_dict[col_name].append(val)
    return data_dict

def get_REopt_data(data_f, scenario_name, config):
    """Fetch and format data for a specific REopt scenario."""
    scenario_name_str = str(scenario_name)
    suffix = "_bau" if re.search(r"(?i)\bBAU\b", scenario_name_str) else ""
    
    df_gen = flatten_dict(data_f)
    data_dict = generate_data_dict(config, df_gen, suffix)
    data_dict["Scenario"] = [scenario_name_str]

    col_order = ["Scenario"] + [col_name for _, col_name in config]
    df_res = pd.DataFrame(data_dict)
    df_res = df_res[col_order]

    return df_res

def get_bau_values(mock_scenarios, config):
    """Retrieve BAU values for comparison."""
    bau_values = {col_name: None for _, col_name in config}
    for scenario in mock_scenarios:
        df_gen = flatten_dict(scenario["outputs"])
        for var_key, col_name in config:
            try:
                key = var_key.__code__.co_consts[1]
            except IndexError:
                print(f"Warning: Could not find constant in lambda for {col_name}. Skipping...")
                continue

            key_bau = f"{key}_bau"
            if key_bau in df_gen:
                value = df_gen[key_bau]
                if bau_values[col_name] is None:
                    bau_values[col_name] = value
                elif bau_values[col_name] != value:
                    raise ValueError(f"Inconsistent BAU values for {col_name}. This should only be used for portfolio cases with the same Site, ElectricLoad, and ElectricTariff for energy consumption and energy costs.")
    return bau_values

def get_scenario_results(run_uuid):
    """Retrieve scenario results from an external API."""
    results_url = f"{root_url}/job/{run_uuid}/results/?api_key={API_KEY}"
    response = requests.get(results_url, verify=False)
    response.raise_for_status()
    result_data = response.json()
    
    processed_data = sum_vectors(result_data) #vectors are summed into a single value
    
    # ## outputs json with the simplified REopt results where vectors are summed into a single value
    # with open(f"{run_uuid}.json", "w") as json_file:
    #     json.dump(processed_data, json_file, indent=4)
    
    return processed_data

def process_scenarios(scenarios, reopt_data_config):
    """Process multiple scenarios and generate a combined dataframe."""
    config = reopt_data_config
    bau_values = get_bau_values(scenarios, config)
    combined_df = pd.DataFrame()
    for scenario in scenarios:
        run_uuid = scenario['run_uuid']
        df_result = get_REopt_data(scenario["outputs"], run_uuid, config)
        df_result = df_result.set_index('Scenario').T
        df_result.columns = [run_uuid]
        combined_df = df_result if combined_df.empty else combined_df.join(df_result, how='outer')

    bau_data = {key: [value] for key, value in bau_values.items()}
    bau_data["Scenario"] = ["BAU"]
    df_bau = pd.DataFrame(bau_data)

    combined_df = pd.concat([df_bau, combined_df.T]).reset_index(drop=True)
    combined_df = clean_data_dict(combined_df.to_dict(orient="list"))
    combined_df = pd.DataFrame(combined_df)
    combined_df = combined_df[["Scenario"] + [col for col in combined_df.columns if col != "Scenario"]]

    return combined_df

def summary_by_runuuids(run_uuids):
    """Fetch summary for multiple run UUIDs."""
    if not run_uuids:
        return {'Error': 'Must provide one or more run_uuids'}

    for r_uuid in run_uuids:
        if not isinstance(r_uuid, str):
            return {'Error': f'Provided run_uuids type error, must be string. {r_uuid}'}
        
        try:
            uuid.UUID(r_uuid)
        except ValueError as e:
            return {"Error": str(e)}
    
    try:
        scenarios = [get_scenario_results(run_uuid) for run_uuid in run_uuids]
        return {'scenarios': scenarios}
    except Exception as e:
        return {"Error": str(e)}

### Main Script Execution (`main`)

- **Retrieves the Scenario Data**: Retrieves results from the REopt API for each scenario.
- **Data Processing**: Processes the scenarios using the functions outlined above.
- **Output Handling**: Converts the processed data into JSON format, and provides options to save it as a CSV file.

In [58]:
# def main():
#     run_uuid_1 = "4043a50f-52b9-482a-90dd-8f7ea417182a"
#     run_uuid_2 = "3ccb973a-e9ed-405e-bb41-4fcb0f4bb9a5"
#     run_uuids = [run_uuid_1, run_uuid_2]

#     scenarios = summary_by_runuuids(run_uuids)
#     if 'scenarios' not in scenarios:
#         print(f"Error: {scenarios['Error']}")
#         return

#     final_df = process_scenarios(scenarios['scenarios'])
#     final_df.iloc[1:, 0] = run_uuids

#     # Transpose the final dataframe
#     final_df_transpose = final_df.transpose()
#     final_df_transpose.columns = final_df_transpose.iloc[0]
#     final_df_transpose = final_df_transpose.drop(final_df_transpose.index[0])
    
#     display(final_df_transpose)

#     def colnum_string(n):
#         """Convert a column number to an Excel-style column string."""
#         string = ""
#         while n > 0:
#             n, remainder = divmod(n - 1, 26)
#             string = chr(65 + remainder) + string
#         return string

#     def create_proforma_with_dataframe(df):
#         # Create a new Excel file and add a worksheet
#         workbook = xlsxwriter.Workbook('ITA_report.xlsx')
#         worksheet = workbook.add_worksheet('ITA Report Template')

#         # Define formats
#         data_format = workbook.add_format({'align': 'center', 'valign': 'center', 'border': 1})
#         formula_format = workbook.add_format({'bg_color': '#a8a8a8', 'align': 'center', 'valign': 'center', 'border': 1, 'font_color': 'red'})
#         # Color the first row and the first column for variable names
#         scenario_header_format = workbook.add_format({'bold': True, 'bg_color': '#D9EAD3', 'border': 1, 'align': 'center'})
#         variable_name_format = workbook.add_format({'bold': True, 'bg_color': '#CFE2F3', 'border': 1, 'align': 'left'})

#         # Set consistent column widths for all columns including the note column
#         column_width = 35  # You can adjust this width as needed
#         for col_num in range(len(df.columns) + 2):  # +2 to account for the 'Note' column
#             worksheet.set_column(col_num, col_num, column_width)
        
#         # Write headers (scenario names)
#         worksheet.write('A1', 'Scenario', scenario_header_format)  # Scenario header
#         for col_num, header in enumerate(df.columns):
#             worksheet.write(0, col_num + 1, header, scenario_header_format)
        
#         # Write variable names
#         for row_num, variable in enumerate(df.index):
#             worksheet.write(row_num + 1, 0, variable, variable_name_format)

#         # Write data
#         for row_num, row_data in enumerate(df.itertuples(index=False)):
#             for col_num, value in enumerate(row_data):
#                 if pd.isnull(value) or value == '-':
#                     worksheet.write(row_num + 1, col_num + 1, "", data_format)
#                 else:
#                     worksheet.write(row_num + 1, col_num + 1, value, data_format)

#         # Map row headers to indices, as the DataFrame is transposed
#         headers = {header: idx for idx, header in enumerate(df.index)}

#         # Fetch the row indices
#         total_site_use_row        = headers['Total Site Electricity Use (kWh)']
#         pv_to_load_row            = headers['PV Serving Load (kWh)']
#         wind_to_load_row          = headers['Wind Serving Load (kWh)']
#         chp_to_load_row           = headers['CHP Serving Load (kWh)']
#         grid_to_load_row          = headers['Grid Purchased Electricity (kWh)']
#         net_elec_reduction_row    = headers['Net Purchased Electricity Reduction (%)']
#         purchased_elec_costs_row  = headers['Purchased Electricity Cost ($)']
#         energy_costs_row          = headers['Electricity Energy Cost ($)']
#         demand_costs_row          = headers['Electricity Demand Cost ($)']
#         fixed_costs_row           = headers['Utility Fixed Cost ($)']
#         net_elec_costs_row        = headers['Net Electricity Cost ($)']
#         export_benefit_row        = headers['Electricity Export Benefit ($)']
#         elec_savings_per_year_row = headers['Electricity Cost Savings ($/year)']
#         total_fuel_row            = headers['Total Fuel (MMBtu)']
#         boiler_fuel_row           = headers['Boiler Fuel (MMBtu)']
#         chp_fuel_row              = headers['CHP Fuel (MMBtu)']
#         ng_reduction_row          = headers['Natural Gas Reduction (%)']
#         total_therm_row           = headers['Total Thermal Production (MMBtu)']
#         boiler_prod_mmbtu_row     = headers['Boiler Thermal Production (MMBtu)']
#         chp_prod_mmbtu_row        = headers['CHP Thermal Production (MMBtu)']
#         ng_costs_row              = headers['Total Fuel (NG) Cost ($)']
#         heat_fuel_costs_row       = headers['Heating System Fuel Cost ($)']
#         chp_fuel_costs_row        = headers['CHP Fuel Cost ($)']
#         total_util_cost_row       = headers['Total Utility Cost ($)']
#         incentive_row             = headers['Incentive Value ($)']
#         gross_cap_cost_row        = headers['Gross Capital Cost ($)']
#         fed_incentive_row         = headers['Federal Tax Incentive (30%)']
#         iac_grant_row             = headers['IAC Grant ($)']
#         net_cap_cost_row          = headers['Net Capital Cost ($)']
#         annual_cost_save_row      = headers['Annual Cost Savings ($)']
#         om_increase_save_row      = headers['O&M Cost Increase ($)']
#         simple_payback_row        = headers['Simple Payback (years)']
#         co2_reduction_row         = headers['CO2 Reduction (tonnes)']
#         co2_emission_row          = headers['CO2 Emissions (tonnes)']
#         co2_savings_row           = headers['CO2 % savings ']

#         # Assuming the BAU (Business As Usual) column is the first data column after the header
#         bau_grid_value_cell          = f'{colnum_string(2)}{grid_to_load_row + 2}'
#         bau_net_cost_value_cell      = f'{colnum_string(2)}{net_elec_costs_row + 2}'
#         bau_ng_reduction_value_cell  = f'{colnum_string(2)}{total_fuel_row + 2}'
#         bau_util_cost_value_cell     = f'{colnum_string(2)}{total_util_cost_row + 2}'
#         bau_co2_reduction_value_cell = f'{colnum_string(2)}{co2_emission_row + 2}'

#         # Write formulas for each row
#         for col in range(2, len(df.columns) + 2):
#             worksheet.write_formula(total_site_use_row + 1, col-1, 
#                                     f'={colnum_string(col)}{pv_to_load_row + 2}+{colnum_string(col)}{wind_to_load_row + 2}+'
#                                     f'{colnum_string(col)}{chp_to_load_row + 2}+{colnum_string(col)}{grid_to_load_row + 2}', formula_format)
            
#             worksheet.write_formula(net_elec_reduction_row + 1, col-1,
#                                     f'=({bau_grid_value_cell}-{colnum_string(col)}{grid_to_load_row + 2})/{bau_grid_value_cell}', formula_format)

#             worksheet.write_formula(purchased_elec_costs_row + 1, col-1, 
#                                     f'={colnum_string(col)}{energy_costs_row + 2}+{colnum_string(col)}{demand_costs_row + 2}+'
#                                     f'{colnum_string(col)}{fixed_costs_row + 2}', formula_format)

#             worksheet.write_formula(net_elec_costs_row + 1, col-1, 
#                                     f'={colnum_string(col)}{purchased_elec_costs_row + 2}-{colnum_string(col)}{export_benefit_row + 2}', formula_format)

#             worksheet.write_formula(elec_savings_per_year_row + 1, col-1,
#                                     f'={bau_net_cost_value_cell}-{colnum_string(col)}{net_elec_costs_row + 2}', formula_format)
            
#             worksheet.write_formula(total_fuel_row + 1, col-1,
#                                     f'={colnum_string(col)}{boiler_fuel_row + 2}+{colnum_string(col)}{chp_fuel_row + 2}', formula_format)
            
#             worksheet.write_formula(ng_reduction_row + 1, col-1,
#                                     f'=({bau_ng_reduction_value_cell}-{colnum_string(col)}{total_fuel_row + 2})/{bau_ng_reduction_value_cell}', formula_format)
            
#             worksheet.write_formula(total_therm_row + 1, col-1,
#                                     f'={colnum_string(col)}{boiler_prod_mmbtu_row + 2}+{colnum_string(col)}{chp_prod_mmbtu_row + 2}', formula_format)
            
#             worksheet.write_formula(ng_costs_row + 1, col-1,
#                                     f'={colnum_string(col)}{heat_fuel_costs_row + 2}+{colnum_string(col)}{chp_fuel_costs_row + 2}', formula_format)
            
#             worksheet.write_formula(total_util_cost_row + 1, col-1,
#                                     f'={colnum_string(col)}{net_elec_costs_row + 2}+{colnum_string(col)}{ng_costs_row + 2}', formula_format)
            
#             worksheet.write_formula(incentive_row + 1, col-1,
#                                     f'={colnum_string(col)}{fed_incentive_row + 2}*{colnum_string(col)}{gross_cap_cost_row + 2}+{colnum_string(col)}{iac_grant_row + 2}', formula_format)
            
#             worksheet.write_formula(net_cap_cost_row + 1, col-1, 
#                                     f'={colnum_string(col)}{gross_cap_cost_row + 2}-{colnum_string(col)}{incentive_row + 2}', formula_format)
            
#             worksheet.write_formula(annual_cost_save_row + 1, col-1,
#                                     f'={bau_util_cost_value_cell}-{colnum_string(col)}{total_util_cost_row + 2}+{colnum_string(col)}{om_increase_save_row + 2}', formula_format)

#             worksheet.write_formula(simple_payback_row + 1, col-1, 
#                                     f'={colnum_string(col)}{net_cap_cost_row + 2}/{colnum_string(col)}{annual_cost_save_row + 2}', formula_format)
            
#             worksheet.write_formula(co2_reduction_row + 1, col-1,
#                                     f'={bau_co2_reduction_value_cell}-{colnum_string(col)}{co2_emission_row + 2}', formula_format)
            
#             worksheet.write_formula(co2_savings_row + 1, col-1,
#                                     f'=({bau_co2_reduction_value_cell}-{colnum_string(col)}{co2_emission_row + 2})/{bau_co2_reduction_value_cell}', formula_format)

#         # Add a note on the far right
#         note_column = len(df.columns) + 2  # One column after the last scenario column
#         worksheet.write(1, note_column, "Values in red are formulas. Do not input anything.", formula_format)

#         # Save the workbook
#         workbook.close()
#         print("Excel file created: ITA_report.xlsx")

#     create_proforma_with_dataframe(final_df_transpose)

# main()


## Custom Table Functionality Overview

The `main` function performs the following tasks:

1. **Initialize UUIDs for Runs**: Sets up the UUIDs for the runs to be analyzed.
2. **Setup Custom Table**: Defines the table structure needed for processing, along with REopt dictionaries to grab data.
3. **Process Scenarios**: Summarizes scenarios based on the run UUIDs.
4. **Create DataFrame**: Processes the scenarios into a DataFrame.65. **Create Proforma**: Generates an Excel report using the transposed DataFrame and specified calculations.

## Detailed Steps

### 1. Initialize UUIDs for Runs

```python
run_uuid_1 = "4043a50f-52b9-482a-90dd-8f7ea417182a"
run_uuid_2 = "3ccb973a-e9ed-405e-bb41-4fcb0f4bb9a5"
run_uuids = [run_uuid_1, run_uuid_2]
```

### 2. Set up custom table
```python
ita_custom_table = [
    (lambda df: get_with_suffix(df, "PV.size_kw", ""), "PV Size (kW)"),
    # Add other fields here
    (lambda df: get_with_suffix(df, "Financial.npv", ""), "NPV"),
]
```

### 3. Optional: Define custom calculations
```python
calculations = [
    {
        "name": "Total Site Electricity Use (kWh)",
        "formula": lambda col, bau, headers: f'={col}{headers["PV Serving Load (kWh)"] + 2}+{col}{headers["Wind Serving Load (kWh)"] + 2}+{col}{headers["CHP Serving Load (kWh)"] + 2}+{col}{headers["Grid Purchased Electricity (kWh)"] + 2}'
    },
    {
        "name": "Net Purchased Electricity Reduction (%)",
        "formula": lambda col, bau, headers: f'=({bau["grid_value"]}-{col}{headers["Grid Purchased Electricity (kWh)"] + 2})/{bau["grid_value"]}'
    },
    {
        "name": "Purchased Electricity Cost ($)",
        "formula": lambda col, bau, headers: f'={col}{headers["Electricity Energy Cost ($)"] + 2}+{col}{headers["Electricity Demand Cost ($)"] + 2}+{col}{headers["Utility Fixed Cost ($)"] + 2}'
    }
]
```

Ensure that if you are referencing `bau` cells in your calculations that you initialize the `variable name` in both the custom table and in `bau_cells`.

In [59]:
def main():
    run_uuid_1 = "4043a50f-52b9-482a-90dd-8f7ea417182a"
    run_uuid_2 = "3ccb973a-e9ed-405e-bb41-4fcb0f4bb9a5"
    run_uuids = [run_uuid_1, run_uuid_2]
    
    # Set up table needed along with REopt dictionaries to grab data 
    ita_custom_table = [
        (lambda df: get_with_suffix(df, "PV.size_kw", ""), "PV Size (kW)"),
        (lambda df: get_with_suffix(df, "Wind.size_kw", ""), "Wind Size (kW)"),
        (lambda df: get_with_suffix(df, "CHP.size_kw", ""), "CHP Size (kW)"),
        (lambda df: get_with_suffix(df, "PV.annual_energy_produced_kwh", ""), "PV Total Electricity Produced (kWh)"),
        (lambda df: get_with_suffix(df, "PV.electric_to_grid_series_kw", ""), "PV Exported to Grid (kWh)"),
        (lambda df: get_with_suffix(df, "PV.electric_to_load_series_kw", ""), "PV Serving Load (kWh)"),
        (lambda df: get_with_suffix(df, "Wind.annual_energy_produced_kwh", ""), "Wind Total Electricity Produced (kWh)"),
        (lambda df: get_with_suffix(df, "Wind.electric_to_grid_series_kw", ""), "Wind Exported to Grid (kWh)"),
        (lambda df: get_with_suffix(df, "Wind.electric_to_load_series_kw", ""), "Wind Serving Load (kWh)"),
        (lambda df: get_with_suffix(df, "CHP.annual_electric_production_kwh", ""), "CHP Total Electricity Produced (kWh)"),
        (lambda df: get_with_suffix(df, "CHP.electric_to_grid_series_kw", ""), "CHP Exported to Grid (kWh)"),
        (lambda df: get_with_suffix(df, "CHP.electric_to_load_series_kw", ""), "CHP Serving Load (kWh)"),
        (lambda df: get_with_suffix(df, "CHP.thermal_to_load_series_mmbtu_per_hour", ""), "CHP Serving Thermal Load (MMBtu)"),
        (lambda df: get_with_suffix(df, "ElectricUtility.annual_energy_supplied_kwh", ""), "Grid Purchased Electricity (kWh)"),
        (lambda df: get_with_suffix(df, "ElectricUtility.electric_to_load_series_kw", ""), "Total Site Electricity Use (kWh)"),
        (lambda df: get_with_suffix(df, "ElectricUtility.electric_to_load_series_kwsdf", ""), "Net Purchased Electricity Reduction (%)"),
        (lambda df: get_with_suffix(df, "ElectricTariff.year_one_energy_cost_before_tax", ""), "Electricity Energy Cost ($)"),
        (lambda df: get_with_suffix(df, "ElectricTariff.year_one_demand_cost_before_tax", ""), "Electricity Demand Cost ($)"),
        (lambda df: get_with_suffix(df, "ElectricTariff.year_one_fixed_cost_before_tax", ""), "Utility Fixed Cost ($)"),
        (lambda df: get_with_suffix(df, "ElectricTariff.year_one_bill_before_tax", ""), "Purchased Electricity Cost ($)"),
        (lambda df: get_with_suffix(df, "ElectricTariff.year_one_export_benefit_before_tax", ""), "Electricity Export Benefit ($)"),
        (lambda df: get_with_suffix(df, "ElectricTariff.lifecycle_energy_cost_after_tax", ""), "Net Electricity Cost ($)"),
        (lambda df: get_with_suffix(df, "ElectricTariff.lifecycle_energy_cost_after_tax_bau", ""), "Electricity Cost Savings ($/year)"),
        (lambda df: get_with_suffix(df, "Boiler.fuel_used_mmbtu", ""), "Boiler Fuel (MMBtu)"),
        (lambda df: get_with_suffix(df, "CHP.annual_fuel_consumption_mmbtu", ""), "CHP Fuel (MMBtu)"),
        (lambda df: get_with_suffix(df, "ElectricUtility.total_energy_supplied_kwh", ""), "Total Fuel (MMBtu)"),
        (lambda df: get_with_suffix(df, "ElectricUtility.annual_energy_supplied_kwh_bau", ""), "Natural Gas Reduction (%)"),
        (lambda df: get_with_suffix(df, "Boiler.annual_thermal_production_mmbtu", ""), "Boiler Thermal Production (MMBtu)"),
        (lambda df: get_with_suffix(df, "CHP.annual_thermal_production_mmbtu", ""), "CHP Thermal Production (MMBtu)"),
        (lambda df: get_with_suffix(df, "CHP.annual_thermal_production_mmbtu", ""), "Total Thermal Production (MMBtu)"),
        (lambda df: get_with_suffix(df, "Site.heating_system_fuel_cost_us_dollars", ""), "Heating System Fuel Cost ($)"),
        (lambda df: get_with_suffix(df, "CHP.year_one_fuel_cost_before_tax", ""), "CHP Fuel Cost ($)"),
        (lambda df: get_with_suffix(df, "Site.total_fuel_cost_us_dollars", ""), "Total Fuel (NG) Cost ($)"),
        (lambda df: get_with_suffix(df, "Site.total_utility_cost_us_dollars", ""), "Total Utility Cost ($)"),
        (lambda df: get_with_suffix(df, "Financial.om_and_replacement_present_cost_after_tax", ""), "O&M Cost Increase ($)"),
        (lambda df: get_with_suffix(df, "Financial.simple_payback_years", ""), "Payback Period (years)"),
        (lambda df: get_with_suffix(df, "Financial.lifecycle_capital_costs", ""), "Gross Capital Cost ($)"),
        (lambda df: get_with_suffix(df, "Financial.total_incentives_us_dollars", ""), "Federal Tax Incentive (30%)"),
        (lambda df: get_with_suffix(df, "Financial.iac_grant_us_dollars", ""), "IAC Grant ($)"),
        (lambda df: get_with_suffix(df, "Financial.total_incentives_value_us_dollars", ""), "Incentive Value ($)"),
        (lambda df: get_with_suffix(df, "Financial.net_capital_cost_us_dollars", ""), "Net Capital Cost ($)"),
        (lambda df: get_with_suffix(df, "Financial.annual_cost_savings_us_dollars", ""), "Annual Cost Savings ($)"),
        (lambda df: get_with_suffix(df, "Financial.simple_payback_years", ""), "Simple Payback (years)"),
        (lambda df: get_with_suffix(df, "Site.annual_emissions_tonnes_CO2", ""), "CO2 Emissions (tonnes)"),
        (lambda df: get_with_suffix(df, "Site.lifecycle_emissions_tonnes_CO2", ""), "CO2 Reduction (tonnes)"),
        (lambda df: get_with_suffix(df, "Site.lifecycle_emissions_tonnes_CO2_bau", ""), "CO2 (%) savings "),
        (lambda df: get_with_suffix(df, "Financial.npv", ""), "NPV"),
    ]

    scenarios = summary_by_runuuids(run_uuids)
    if 'scenarios' not in scenarios:
        print(f"Error: {scenarios['Error']}")
        return

    final_df = process_scenarios(scenarios['scenarios'], ita_custom_table)
    final_df.iloc[1:, 0] = run_uuids

    # Transpose the final dataframe
    final_df_transpose = final_df.transpose()
    final_df_transpose.columns = final_df_transpose.iloc[0]
    final_df_transpose = final_df_transpose.drop(final_df_transpose.index[0])
    
    display(final_df_transpose)

    def colnum_string(n):
        """Convert a column number to an Excel-style column string."""
        string = ""
        while n > 0:
            n, remainder = divmod(n - 1, 26)
            string = chr(65 + remainder) + string
        return string

    def create_proforma_with_dataframe(df, custom_table, calculations):
        # Create a new Excel file and add a worksheet
        workbook = xlsxwriter.Workbook('ITA_report.xlsx')
        worksheet = workbook.add_worksheet('ITA Report Template')

        # Define formats
        data_format            =   workbook.add_format({'align': 'center', 'valign': 'center', 'border': 1})
        formula_format         =   workbook.add_format({'bg_color': '#C1EE86', 'align': 'center', 'valign': 'center', 'border': 1, 'font_color': 'red'})
        scenario_header_format =   workbook.add_format({'bold': True, 'bg_color': '#0079C2', 'border': 1, 'align': 'center', 'font_color': 'white'})
        variable_name_format   =   workbook.add_format({'bold': True, 'bg_color': '#DEE2E5', 'border': 1, 'align': 'left'})

        # Note
        worksheet.write(1, len(df.columns) + 2, "Values in red are formulas. Do not input anything.", formula_format)

        # Set consistent column widths for all columns including the note column
        column_width = 35
        for col_num in range(len(df.columns) + 3):  # +3 to account for the 'Note' column
            worksheet.set_column(col_num, col_num, column_width)
        
        # Write headers (scenario names)
        worksheet.write('A1', 'Scenario', scenario_header_format)  # Scenario header
        for col_num, header in enumerate(df.columns):
            worksheet.write(0, col_num + 1, header, scenario_header_format)
        
        # Write variable names
        for row_num, variable in enumerate(df.index):
            worksheet.write(row_num + 1, 0, variable, variable_name_format)

        # Write data
        for row_num, row_data in enumerate(df.itertuples(index=False)):
            for col_num, value in enumerate(row_data):
                worksheet.write(row_num + 1, col_num + 1, "" if pd.isnull(value) or value == '-' else value, data_format)

        # Map row headers to indices, as the DataFrame is transposed
        headers = {header: idx for idx, header in enumerate(df.index)}

        # BAU cell references
        bau_cells = {
            'grid_value': f'{colnum_string(2)}{headers["Grid Purchased Electricity (kWh)"] + 2}' if "Grid Purchased Electricity (kWh)" in headers else None,
            'net_cost_value': f'{colnum_string(2)}{headers["Net Electricity Cost ($)"] + 2}' if "Net Electricity Cost ($)" in headers else None,
            'ng_reduction_value': f'{colnum_string(2)}{headers["Total Fuel (MMBtu)"] + 2}' if "Total Fuel (MMBtu)" in headers else None,
            'util_cost_value': f'{colnum_string(2)}{headers["Total Utility Cost ($)"] + 2}' if "Total Utility Cost ($)" in headers else None,
            'co2_reduction_value': f'{colnum_string(2)}{headers["CO2 Emissions (tonnes)"] + 2}' if "CO2 Emissions (tonnes)" in headers else None
        }

        # Write formulas based on the configuration
        missing_entries = []
        for col in range(2, len(df.columns) + 2):
            col_letter = colnum_string(col)
            for calc in calculations:
                if calc["name"] in headers:
                    row_idx = headers[calc["name"]]
                    formula = calc["formula"](col_letter, bau_cells, headers)
                    if formula:  # Ensure the formula is not None
                        worksheet.write_formula(row_idx + 1, col-1, formula, formula_format)
                    else:
                        missing_entries.append(calc["name"])
                else:
                    missing_entries.append(calc["name"])

        if missing_entries:
            print(f"Missing entries in the input table: {', '.join(set(missing_entries))}. Please update the configuration if necessary.")

        workbook.close()
        print("Excel file created: ITA_report.xlsx")


    # Configuration for calculations
    calculations = [
        {
            "name": "Total Site Electricity Use (kWh)",
            "formula": lambda col, bau, headers: f'={col}{headers["PV Serving Load (kWh)"] + 2}+{col}{headers["Wind Serving Load (kWh)"] + 2}+{col}{headers["CHP Serving Load (kWh)"] + 2}+{col}{headers["Grid Purchased Electricity (kWh)"] + 2}'
        },
        {
            "name": "Net Purchased Electricity Reduction (%)",
            "formula": lambda col, bau, headers: f'=({bau["grid_value"]}-{col}{headers["Grid Purchased Electricity (kWh)"] + 2})/{bau["grid_value"]}'
        },
        {
            "name": "Purchased Electricity Cost ($)",
            "formula": lambda col, bau, headers: f'={col}{headers["Electricity Energy Cost ($)"] + 2}+{col}{headers["Electricity Demand Cost ($)"] + 2}+{col}{headers["Utility Fixed Cost ($)"] + 2}'
        },
        {
            "name": "Net Electricity Cost ($)",
            "formula": lambda col, bau, headers: f'={col}{headers["Purchased Electricity Cost ($)"] + 2}-{col}{headers["Electricity Export Benefit ($)"] + 2}'
        },
        {
            "name": "Electricity Cost Savings ($/year)",
            "formula": lambda col, bau, headers: f'={bau["net_cost_value"]}-{col}{headers["Net Electricity Cost ($)"] + 2}'
        },
        {
            "name": "Total Fuel (MMBtu)",
            "formula": lambda col, bau, headers: f'={col}{headers["Boiler Fuel (MMBtu)"] + 2}+{col}{headers["CHP Fuel (MMBtu)"] + 2}'
        },
        {
            "name": "Natural Gas Reduction (%)",
            "formula": lambda col, bau, headers: f'=({bau["ng_reduction_value"]}-{col}{headers["Total Fuel (MMBtu)"] + 2})/{bau["ng_reduction_value"]}'
        },
        {
            "name": "Total Thermal Production (MMBtu)",
            "formula": lambda col, bau, headers: f'={col}{headers["Boiler Thermal Production (MMBtu)"] + 2}+{col}{headers["CHP Thermal Production (MMBtu)"] + 2}'
        },
        {
            "name": "Total Fuel (NG) Cost ($)",
            "formula": lambda col, bau, headers: f'={col}{headers["Heating System Fuel Cost ($)"] + 2}+{col}{headers["CHP Fuel Cost ($)"] + 2}'
        },
        {
            "name": "Total Utility Cost ($)",
            "formula": lambda col, bau, headers: f'={col}{headers["Net Electricity Cost ($)"] + 2}+{col}{headers["Total Fuel (NG) Cost ($)"] + 2}'
        },
        {
            "name": "Incentive Value ($)",
            "formula": lambda col, bau, headers: f'={col}{headers["Federal Tax Incentive (30%)"] + 2}*{col}{headers["Gross Capital Cost ($)"] + 2}+{col}{headers["IAC Grant ($)"] + 2}'
        },
        {
            "name": "Net Capital Cost ($)",
            "formula": lambda col, bau, headers: f'={col}{headers["Gross Capital Cost ($)"] + 2}-{col}{headers["Incentive Value ($)"] + 2}'
        },
        {
            "name": "Annual Cost Savings ($)",
            "formula": lambda col, bau, headers: f'={bau["util_cost_value"]}-{col}{headers["Total Utility Cost ($)"] + 2}+{col}{headers["O&M Cost Increase ($)"] + 2}'
        },
        {
            "name": "Simple Payback (years)",
            "formula": lambda col, bau, headers: f'={col}{headers["Net Capital Cost ($)"] + 2}/{col}{headers["Annual Cost Savings ($)"] + 2}'
        },
        {
            "name": "CO2 Reduction (tonnes)",
            "formula": lambda col, bau, headers: f'={bau["co2_reduction_value"]}-{col}{headers["CO2 Emissions (tonnes)"] + 2}'
        },
        {
            "name": "CO2 (%) savings ",
            "formula": lambda col, bau, headers: f'=({bau["co2_reduction_value"]}-{col}{headers["CO2 Emissions (tonnes)"] + 2})/{bau["co2_reduction_value"]}'
        }
    ]
    
    create_proforma_with_dataframe(final_df_transpose, ita_custom_table, calculations)

main()


Scenario,BAU,4043a50f-52b9-482a-90dd-8f7ea417182a,3ccb973a-e9ed-405e-bb41-4fcb0f4bb9a5
PV Size (kW),,2837.237,3293.6635
Wind Size (kW),,583.38,
CHP Size (kW),,273.931431,273.931431
PV Total Electricity Produced (kWh),,4244701.21,4927546.14
PV Exported to Grid (kWh),,,
PV Serving Load (kWh),,2935677.738,3247900.461
Wind Total Electricity Produced (kWh),,1285854.0,
Wind Exported to Grid (kWh),,,
Wind Serving Load (kWh),,997809.852,
CHP Total Electricity Produced (kWh),,1699558.091,1797415.524


Excel file created: ITA_report.xlsx
